In [21]:
import numpy as np
import math

In [ ]:
# state space:
# coordinates relative to checkpoints coordinates C = (x, y)
# velocity relative to C
# acceleration relative to C
# angular velocity relativ to global coordinate system
# angualar acceleration relativ to global coordinate system
# difference vector between optimal direction of drone body and current direction (normalised unit vectors)

class DroneEnv:
    def __init__(self, width=1000, height=1000, checkpoint_radius=30):
        # environment
        self.width = width
        self.height = height
        self.checkpoint_radius = checkpoint_radius

        # environment params:
        self.GRAVITY = 2.5
        self.THRUST_POWER = 5
        self.X_AXIS_SENS = 50
        self.Y_AXIS_SENS = 1
        self.ROTATION_SPEED = 10
        self.ROTATION_DRAG = 0.7
        self.MOVEMENT_DRAG = 0.5

        # initiate drone
        self.reset()

    def reset(self, x, y, vx, vy, ax, ay, a, va, aa):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.ax = ax
        self.ay = ay
        self.angle = a # in degrees
        self.va = va
        self.aa = aa

        self.spawn_checkpoint()

        return self.get_state() # not likely to be of use

    def spawn_checkpoint(self):
        self.checkpoint_x = np.random.uniform(0.1 * self.width, 0.9 * self.width)
        self.checkpoint_y = np.random.uniform(0.1 * self.height, 0.9 * self.height)

    def step(self, action):
        thrust_left = action[0] # thrust values
        thrust_right = action[1]

        # physics:
        self.vy += self.GRAVITY # Y

        torque = thrust_left - thrust_right # factor

        self.angular_velocity -= self.ROTATION_SPEED * torque # VA
        thrust = self.THRUST_POWER * math.cos(math.radians(self.angle))
        self.vy -= self.Y_AXIS_SENS * thrust # Y

        self.angular_velocity *= self.ROTATION_DRAG

        self.angle += self.angular_velocity
        self.angle %= 360.0

        angle_rad = math.radians(self.angle)
        self.vx = self.X_AXIS_SENS * math.sin(angle_rad)

        self.x += self.MOVEMENT_DRAG * self.vx
        self.y += self.MOVEMENT_DRAG * self.vy

        dx = self.checkpoint_x - self.x
        dy = self.checkpoint_y - self.y
        dist = math.sqrt(dx * dx + dy * dy)

        reached = False
        reward = -dist * 0.01

        if dist < self.checkpoint_radius:
            reached = True
            reward += 10.0
            self.spawn_checkpoint()

        done = False
        if self.x < 0 or self.x > self.width or self.y < 0 or self.y > self.height:
            done = True
            reward -= 5.0

        return self.get_state(), reward, done, {"checkpoint_reached": reached}

    def get_state(self):
        dx = self.checkpoint_x - self.x
        dy = self.checkpoint_y - self.y

        return np.array([
            self.x / self.width,
            self.y / self.height,
            self.vx / self.X_AXIS_SENS,
            self.vy / 100,
            math.sin(math.radians(self.angle)),
            math.cos(math.radians(self.angle)),
            self.angular_velocity / self.ROTATION_SPEED,
            dx / self.width,
            dy / self.height,
        ], dtype=np.float32)


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = torch.device("cuda")
torch.cuda.is_available()

#print(torch.version.cuda)


True